## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2024xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [1]:
from datetime import datetime
import json
from tab_dataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib

def analyse_integrite(data, schema, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le schéma 'schema'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement affiché (paramètre 'affiche=True') . 
    Les lignes en erreur sont optionnellement ajoutées (paramètre 'indic=True') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(schema)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc'
    Le nombre d'écarts est affiché optionnellement (paramètre 'affiche=True').
    '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    nbre_pdc = data['nbre_pdc'].astype('int')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == nbre_pdc
    data['calc-nbre'] = data['nb_pdc_calc'] - nbre_pdc
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [2]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15', '2024-02-07', '2024-02-28', '2024-03-27', '2024-04-24', '2024-05-19', '2024-06-05', '2024-08-02']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.3.1-'+date[:4]+date[5:7]+date[8:]+'.csv', # après 2024-02-06
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
      }
irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False, dtype='object')
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  107939


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [3]:
# complément à inclure dans le schéma de données
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         #{"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" }
    ]
}
# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 3 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']] = data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  3175 soit :  0.029414762041523452
nombre de pdc en itinerance   :  104764


In [6]:
# arborescence des champs et nombre de valeurs différentes
print(itinerance_init.npd.analysis().tree())

-1: root-derived (104764)
   0 : index (0 - 104764)
   1 : contact_operateur (104590 - 174)
   2 : nom_enseigne (101389 - 3375)
   3 : coordonneesXY (77046 - 27718)
   4 : adresse_station (77051 - 27713)
   5 : id_station_itineranc (62097 - 42667)
   6 : nom_station (76454 - 28310)
   7 : implantation_station (104759 - 5)
   8 : nbre_pdc (104699 - 65)
   9 : condition_acces (104762 - 2)
   10: horaires (103983 - 781)
   11: station_deux_roues (104758 - 6)
   12: id_pdc_itinerance (8085 - 96679)
   13: date_maj (103963 - 801)
   14: last_modified (104194 - 570)


-----------------------
## 4 - Bilan initial intégrité
- 22 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, schema)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          16168
contact_operateur - id_station_itinerance           7488
nom_enseigne - id_station_itinerance                8101
coordonneesXY - id_station_itinerance               9015
id_station_itinerance - id_pdc_itinerance           8168
nom_station - id_station_itinerance                 8876
implantation_station - id_station_itinerance        3624
nbre_pdc - id_station_itinerance                    3360
condition_acces - id_station_itinerance               83
horaires - id_station_itinerance                      53
station_deux_roues - id_station_itinerance          8227
adresse_station - coordonneesXY                     4748

nombre d'enregistrements sans erreurs :  82645
nombre d'enregistrements avec au moins une erreur :  22119
taux d'erreur :  21  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 35% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  8085  soit  37  %
nombre de pdc sans doublon   :  14034  soit  63  %


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, schema)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             85
nom_enseigne - id_station_itinerance                  88
coordonneesXY - id_station_itinerance                955
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1452
implantation_station - id_station_itinerance         188
nbre_pdc - id_station_itinerance                    1075
condition_acces - id_station_itinerance               20
horaires - id_station_itinerance                      30
station_deux_roues - id_station_itinerance           847
adresse_station - coordonneesXY                     4474

nombre d'enregistrements sans erreurs :  8008
nombre d'enregistrements avec au moins une erreur :  6026


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 43 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  1420  soit  6  %
nombre de pdc sans doublon   :  4606


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, schema)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                   5
coordonneesXY - id_station_itinerance                367
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  395
implantation_station - id_station_itinerance         152
nbre_pdc - id_station_itinerance                     364
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      27
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     2034

nombre d'enregistrements sans erreurs :  1356
nombre d'enregistrements avec au moins une erreur :  3250  soit  54  %


In [12]:
# structure des données présentant une erreur et nombre de valeurs
print(itinerance_5.npd.analysis().tree())

-1: root-derived (3250)
   0 : index (0 - 3250)
   1 : contact_operateur (3197 - 53)
   2 : nom_enseigne (3057 - 193)
      9 : condition_acces (191 - 2)
   3 : coordonneesXY (2643 - 607)
   4 : adresse_station (2286 - 964)
   5 : id_station_itineranc (1775 - 1475)
      13: date_maj (1364 - 111)
   6 : nom_station (2355 - 895)
   7 : implantation_station (3245 - 5)
   8 : nbre_pdc (3220 - 30)
   10: horaires (3188 - 62)
   11: station_deux_roues (3244 - 6)
   12: id_pdc_itinerance (0 - 3250)
   14: last_modified (3158 - 92)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  3  erreurs :  142


In [14]:
# principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op_5, oper_5 = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))

print(len(err_op_5), ' opérateurs représentent : ', round(sum(err_op_5) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(err_op_5, oper_5):
    print(err, op)

5  opérateurs représentent :  67  % des écarts

666 support@alizecharge.fr
600 supervision-ev.france@totalenergies.com
321 sav@izivia.com
302 info@ionity.eu
286 a.buffler@bouygues-es.com


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [15]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  3250
total des doublons à supprimer :  9505
nombre de pdc avec controles ok :  92009


In [16]:
# génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

In [17]:
l_prefix = list(pd.read_csv(log['chemin'] + '/' + 'Operations_Recharge_16-07-2024.csv', sep=';', low_memory=False, dtype='object')['Identifiant'])
l_prefix[0]

'FR073'

In [18]:
import numpy as np

df = pd.DataFrame({'a':["er", "rt", "", "ert"], 'b': [10, np.nan, 20, 1]})

df['b'] = df['b'].replace(np.nan, 0)
df['a'] = df['a'].replace("", "empty")
df
itinerance_ok['cable_t2_attache'].astype('bool')


0        True
1        True
2        True
3        True
4        True
         ... 
92004    True
92005    True
92006    True
92007    True
92008    True
Name: cable_t2_attache, Length: 92009, dtype: bool

In [19]:
from pydantic import (BaseModel, EmailStr, PastDate, PositiveFloat, PositiveInt, ValidationError)
from pydantic_extra_types.phone_numbers import PhoneNumber
from copy import copy
import re

PREFIX = list(pd.read_csv(log['chemin'] + '/' + 'Operations_Recharge_16-07-2024.csv', sep=';', low_memory=False, dtype='object')['Identifiant'])
    
NUMPDL = re.compile(r"^\d{14}$")

class FrenchPhoneNumber(PhoneNumber):
    """A phone number with french defaults."""
    default_region_code = "FR"
    
class EmailStr(BaseModel):
    x: EmailStr

class PastDate(BaseModel):
    x: PastDate

class PositiveInt(BaseModel):
    x: PositiveInt

class FrenchPhoneNumber(BaseModel):
    x: FrenchPhoneNumber

class NumPdl:
    def __init__(self, x=None):
        self.x = x
        if NUMPDL.fullmatch(x) is None:
            raise ValidationError

def check_value(value, clas=None, default=None):    
    try:
        clas(x=value)
    except:
        return default
    return value

def set_prefix(pref):    
    return pref if pref in PREFIX else PREFIX[0]
    
def pandas_to_qualicharge(data):
    '''
    convertit les champs suivants (et applique une valeur par défaut si erreur) :
        - contact_amenageur: EmailStr ('amenageur@email.fr')
        - contact_operateur: EmailStr ('example@email.fr')
        - telephone_operateur: FrenchPhoneNumber (06-06-06-06-06)
        - nbre_pdc: PositiveInt (0)
        - date_maj: PastDate ('1900-01-01')
        - num_pdl: "^\d{14}$" ('00001234567890')
        - date_mise_en_service: PastDate ('1900-01-01')
        - puissance_nominale: PositiveFloat (0.0)
    et applique le prefixe des stations aux pdc.
    '''
    quali = copy(data)

    quali['paiement_autre'] = quali['paiement_autre'].astype('bool')
    quali['cable_t2_attache'] = quali['cable_t2_attache'].astype('bool')
    quali['raccordement'] = quali['raccordement'].replace(np.nan, 'Direct')
    quali['siren_amenageur'] = quali['siren_amenageur'].astype('string').replace(["", np.nan], "000000000")
    quali['code_insee_commune'] = quali['code_insee_commune'].astype('string').replace(["", np.nan], "99999")
    
    quali['contact_amenageur'] = quali['contact_amenageur'].apply(check_value,  clas=EmailStr, default='amenageur@email.fr')
    quali['contact_operateur'] = quali['contact_operateur'].apply(check_value,  clas=EmailStr, default='operateur@email.fr')
    quali['telephone_operateur'] = quali['telephone_operateur'].apply(check_value,  clas=FrenchPhoneNumber, default='06-06-06-06-06')
    quali['nbre_pdc'] = quali['nbre_pdc'].apply(check_value,  clas=PositiveInt, default=0)
    quali['date_maj'] = quali['date_maj'].apply(check_value,  clas=PastDate, default='1900-01-01')
    quali['num_pdl'] = quali['num_pdl'].apply(check_value,  clas=NumPdl, default='00001234567890')
    quali['date_mise_en_service'] = quali['date_mise_en_service'].apply(check_value,  clas=PastDate, default='1900-01-01')
    quali['puissance_nominale'] = quali['puissance_nominale'].apply(check_value,  clas=PositiveFloat, default=0.01)
    prefix = quali['id_station_itinerance'].astype('str').str.slice(stop=5).apply(set_prefix)
    quali['id_station_itinerance'] = prefix.str.cat(quali['id_station_itinerance'].astype('str').str.slice(start=5))
    quali['id_pdc_itinerance'] = prefix.str.cat(quali['id_pdc_itinerance'].astype('str').str.slice(start=5))

    quali['restriction_gabarit'] = quali['restriction_gabarit'].astype('string')
    return quali

itinerance_quali = pandas_to_qualicharge(itinerance_ok)

In [20]:
type(itinerance_quali.loc[85, 'restriction_gabarit'])
itinerance_quali['paiement_autre'].unique()

array([ True])

In [21]:
itinerance_quali.loc[:80, relations]


,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,0,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER1,2023-11-05,2024-08-01T03:01:18+00:00
1,1,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER10,2023-11-05,2024-08-01T03:01:18+00:00
2,2,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER11,2023-11-05,2024-08-01T03:01:18+00:00
3,3,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER12,2023-11-05,2024-08-01T03:01:18+00:00
4,4,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER2,2023-11-05,2024-08-01T03:01:18+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,76,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P558842825981017213,Zunder/122289,Parking public,38,Accès libre,24/7,False,FR073EFR3701ER33,2024-06-20,2024-08-01T03:01:18+00:00
77,77,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P558842825981017213,Zunder/122289,Parking public,38,Accès libre,24/7,False,FR073EFR3701ER4,2024-06-20,2024-08-01T03:01:18+00:00
78,78,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P558842825981017213,Zunder/122289,Parking public,38,Accès libre,24/7,False,FR073EFR3701EV1,2024-06-20,2024-08-01T03:01:18+00:00
79,79,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P558842825981017213,Zunder/122289,Parking public,38,Accès libre,24/7,False,FR073EFR3701EV3,2024-06-20,2024-08-01T03:01:18+00:00


In [22]:
itinerance_quali.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,0,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER1,2023-11-05,2024-08-01T03:01:18+00:00
1,1,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER10,2023-11-05,2024-08-01T03:01:18+00:00
2,2,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER11,2023-11-05,2024-08-01T03:01:18+00:00
3,3,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER12,2023-11-05,2024-08-01T03:01:18+00:00
4,4,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P1710169343692745903,Zunder/74654,Parking public,12,Accès libre,24/7,False,FR073E1111ER2,2023-11-05,2024-08-01T03:01:18+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92004,107822,advenir@zeborne.com,TOYOTA NICE - NOVELLIPSE,"[7.201547, 43.6885902]",108 Avenue Simone Veil 06200 Nice,FRZTLE24DC55361,Toyota - Nice - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:30,Mo-Fr 13:30–19:00,Sat 09:00-...",false,FRZTLE24DC55361,2021-02-24,2024-01-19T07:46:20.729000+00:00
92005,107823,advenir@zeborne.com,GARAGE NELLO CHELI - BEAUNE,"[4.86082418, 47.0236522]",RUE LUCIEN PERRIAUX ZI DE VIGNOLES 21200 Beaune,FRZTLE24DC61318,Toyota - Beaune - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE24DC61318,2021-05-24,2024-01-19T07:46:19.511000+00:00
92006,107824,advenir@zeborne.com,GARAGE NELLO CHELI - BEAUNE,"[4.86082418, 47.0236522]",RUE LUCIEN PERRIAUX ZI DE VIGNOLES 21200 Beaune,FRZTLE24DC61319,Toyota - Beaune - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE24DC61319,2021-05-24,2024-01-19T07:46:19.511000+00:00
92007,107825,advenir@zeborne.com,TOYOTA ANNEMASSE - DEGENEVE,"[6.2174783, 46.1858677]",8 RUE DE L'INDUSTRIE 74240 Gaillard,FRZTLE24DC63446,Toyota - Annemasse - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-19:00,Sat 09:00-...",false,FRZTLE24DC63446,2021-06-16,2024-01-19T07:46:18.987000+00:00


In [23]:
import csv
itinerance_quali.to_csv('IRVE_itinerance_valide_quali' + extension, index=False, quoting=csv.QUOTE_ALL, na_rep='empty')

In [24]:
itinerance_quali['paiement_autre'].to_csv('test.csv', index=False, quoting=csv.QUOTE_ALL, na_rep='empty')

In [25]:
for op in oper_5:
    print('IRVE_itinerance_residuel' + '_' + op + '_' + extension)
    itinerance_5_full.loc[itinerance_5_full.contact_operateur == op].to_csv('IRVE_itinerance_residuel' + '_' + op + '_' + extension)

IRVE_itinerance_residuel_support@alizecharge.fr_2024-08-02.csv
IRVE_itinerance_residuel_supervision-ev.france@totalenergies.com_2024-08-02.csv
IRVE_itinerance_residuel_sav@izivia.com_2024-08-02.csv
IRVE_itinerance_residuel_info@ionity.eu_2024-08-02.csv
IRVE_itinerance_residuel_a.buffler@bouygues-es.com_2024-08-02.csv


### vérification de l'intégrité

In [26]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], schema, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [27]:
# structure des données bonnes
#print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())
print(itinerance_ok.loc[:, mandatory].npd.analysis().tree())

-1: root-derived (92009)
   4 : id_station_itineranc (52039 - 39970)
      0 : contact_operateur (39812 - 158)
      1 : nom_enseigne (36745 - 3225)
      2 : coordonneesXY (15379 - 24591)
         3 : adresse_station (483 - 24108)
      5 : nom_station (15168 - 24802)
      6 : implantation_station (39965 - 5)
      7 : nbre_pdc (39905 - 65)
      8 : condition_acces (39968 - 2)
      9 : horaires (39237 - 733)
      10: station_deux_roues (39964 - 6)
      18: paiement_acte (39962 - 8)
      22: restriction_gabarit (39867 - 103)
   11: id_pdc_itinerance (0 - 92009)
   12: puissance_nominale (91846 - 163)
   13: prise_type_ef (92001 - 8)
   14: prise_type_2 (92001 - 8)
   15: prise_type_combo_ccs (92002 - 7)
   16: prise_type_chademo (92002 - 7)
   17: prise_type_autre (92002 - 7)
   19: paiement_autre (92000 - 9)
   20: reservation (92002 - 7)
   21: accessibilite_pmr (92005 - 4)
   23: date_maj (91351 - 658)
   24: last_modified (91514 - 495)


### Indicateurs

In [28]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)

In [29]:
# ajout des principaux opérateurs contributeur de défauts
indic['operateurs'] = list(oper_5)
indic['erreurs_operateurs'] = list(err_op_5)

In [30]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           1281
39  %


In [31]:
# indicateur d'écart entre nombre de pdc calculés et saisis sur les données sans doublon
itinerance_sans_doublons = itinerance[~itinerance['doublons_a_supprimer']].reset_index(drop=True)
add_nbre_pdc(itinerance_sans_doublons, affiche=True)

nbre_pdc_ko          30487
32  %


In [32]:
len(itinerance_4)

4606

In [33]:
# stockage des indicateurs
log |= indic
log_file = log['chemin'] + '/' + 'logfile.txt'
#with open('logfile.txt', 'a', encoding="utf-8") as f:
with open(log_file, 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

In [34]:
itinerance_5_full.columns[16:28]

Index(['index - id_pdc_itinerance',
       'contact_operateur - id_station_itinerance',
       'nom_enseigne - id_station_itinerance',
       'coordonneesXY - id_station_itinerance',
       'id_station_itinerance - id_pdc_itinerance',
       'nom_station - id_station_itinerance',
       'implantation_station - id_station_itinerance',
       'nbre_pdc - id_station_itinerance',
       'condition_acces - id_station_itinerance',
       'horaires - id_station_itinerance',
       'station_deux_roues - id_station_itinerance',
       'adresse_station - coordonneesXY'],
      dtype='object')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [35]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
107,2089,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790186,ORY - Parking P3 SILO - Niveau 5,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901860,2024-07-26,2024-07-26T08:27:18.771000+00:00
108,2090,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790186,ORY - Parking P3 SILO - Niveau 5,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901861,2024-07-26,2024-07-26T08:27:18.771000+00:00
109,2091,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790186,ORY - Parking P3 SILO - Niveau 5,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901862,2024-07-26,2024-07-26T08:27:18.771000+00:00
110,2092,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790186,ORY - Parking P3 SILO - Niveau 5,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901863,2024-07-26,2024-07-26T08:27:18.771000+00:00
111,2093,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790186,ORY - Parking P3 SILO - Niveau 5,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901864,2024-07-26,2024-07-26T08:27:18.771000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,99464,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550401,2024-07-26,2024-07-26T08:27:18.771000+00:00
3980,99465,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550402,2024-07-26,2024-07-26T08:27:18.771000+00:00
3981,99466,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550403,2024-07-26,2024-07-26T08:27:18.771000+00:00
3982,99467,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550404,2024-07-26,2024-07-26T08:27:18.771000+00:00


In [36]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [37]:
print('PdC avec une incohérence : ')
for operateur in oper_5:
    print('\n', sum(itinerance_5.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

PdC avec une incohérence : 

 666
Répartition des incohérences  support@alizecharge.fr  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                197
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  264
implantation_station - id_station_itinerance          35
nbre_pdc - id_station_itinerance                     128
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                       0
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                      409

 600
Répartition des incohérences  supervision-ev.france@totalenergies.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinera

### Defauts operateur Ouestcharge-Paysdelaloire
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [38]:
sum(itinerance_sans_doublons.contact_operateur == 'assistance-technique@ouestcharge-paysdelaloire-moncompte.fr')

536

In [39]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'assistance-technique@ouestcharge-paysdelaloire-moncompte.fr', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [40]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.397512, 47.074964]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Defauts operateur ALIZECHARGE
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [41]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations][50:200]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
94,2076,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790184,ORY - Parking P3 SILO - Niveau 4,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901847,2024-07-26,2024-07-26T08:27:18.771000+00:00
95,2077,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790184,ORY - Parking P3 SILO - Niveau 4,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901848,2024-07-26,2024-07-26T08:27:18.771000+00:00
96,2078,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790184,ORY - Parking P3 SILO - Niveau 5,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901849,2024-07-26,2024-07-26T08:27:18.771000+00:00
97,2079,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790185,2024-07-26,2024-07-26T08:27:18.771000+00:00
98,2080,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790185,ORY - Parking P3 SILO - Niveau 5,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901850,2024-07-26,2024-07-26T08:27:18.771000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,62355,support@alizecharge.fr,EFFIA France,"[3.172768768,50.6785353]","Boulevard Lacordaire, 59100 ROUBAIX",FRP01E595120011,Parking CH Roubaix P3 urgence - EFFIA,Parking public,9,Accès libre,Mo-Su 00:00-23:57,False,FRP01E59512001131,2024-07-26,2024-07-26T08:27:18.771000+00:00
2624,62356,support@alizecharge.fr,EFFIA France,"[3.172768768,50.6785353]","Boulevard Lacordaire, 59100 ROUBAIX",FRP01E595120011,Parking CH Roubaix P3 urgence - EFFIA,Parking public,9,Accès libre,Mo-Su 00:00-23:57,False,FRP01E59512001141,2024-07-26,2024-07-26T08:27:18.771000+00:00
2625,62357,support@alizecharge.fr,EFFIA France,"[3.172768768,50.6785353]","Boulevard Lacordaire, 59100 ROUBAIX",FRP01E595120011,Parking CH Roubaix P3 urgence - EFFIA,Parking public,9,Accès libre,Mo-Su 00:00-23:57,False,FRP01E59512001151,2024-07-26,2024-07-26T08:27:18.771000+00:00
2626,62358,support@alizecharge.fr,EFFIA France,"[3.172768768,50.6785353]","Boulevard Lacordaire, 59100 ROUBAIX",FRP01E595120011,Parking CH Roubaix P3 urgence - EFFIA,Parking public,9,Accès libre,Mo-Su 00:00-23:57,False,FRP01E59512001161,2024-07-26,2024-07-26T08:27:18.771000+00:00


In [42]:
# example [2.362554, 48.728787]
adp = itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]']
adresses = list(adp.drop_duplicates('adresse_station')['adresse_station'])
print(adresses)

print(adp.loc[~adp['nom_station - id_station_itinerance'], ['nom_station', 'id_station_itinerance']])

print(adp.loc[~adp['implantation_station - id_station_itinerance'], ['implantation_station', 'id_station_itinerance']])

print(adp.loc[~adp['nbre_pdc - id_station_itinerance'], ['nbre_pdc', 'id_station_itinerance']])

itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]', relations]

[]
Empty DataFrame
Columns: [nom_station, id_station_itinerance]
Index: []
Empty DataFrame
Columns: [implantation_station, id_station_itinerance]
Index: []
Empty DataFrame
Columns: [nbre_pdc, id_station_itinerance]
Index: []


,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [43]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations].loc[~itinerance_4['adresse_station - coordonneesXY']][500:600]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [44]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FRADPE914790181', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
54,2036,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901810,2024-07-26,2024-07-26T08:27:18.771000+00:00
55,2037,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901811,2024-07-26,2024-07-26T08:27:18.771000+00:00
56,2038,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901812,2024-07-26,2024-07-26T08:27:18.771000+00:00
57,2039,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901813,2024-07-26,2024-07-26T08:27:18.771000+00:00
58,2040,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901814,2024-07-26,2024-07-26T08:27:18.771000+00:00
59,2041,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901815,2024-07-26,2024-07-26T08:27:18.771000+00:00
60,2042,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901816,2024-07-26,2024-07-26T08:27:18.771000+00:00
61,2043,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 3,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901817,2024-07-26,2024-07-26T08:27:18.771000+00:00
62,2044,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 3,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901818,2024-07-26,2024-07-26T08:27:18.771000+00:00
63,2045,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 3,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901819,2024-07-26,2024-07-26T08:27:18.771000+00:00


In [45]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FRADPE914790181', controles]

,index - id_pdc_itinerance,contact_operateur - id_station_itinerance,nom_enseigne - id_station_itinerance,coordonneesXY - id_station_itinerance,id_station_itinerance - id_pdc_itinerance,nom_station - id_station_itinerance,implantation_station - id_station_itinerance,nbre_pdc - id_station_itinerance,condition_acces - id_station_itinerance,horaires - id_station_itinerance,station_deux_roues - id_station_itinerance,adresse_station - coordonneesXY
54,True,True,True,True,True,False,True,True,True,True,True,False
55,True,True,True,True,True,False,True,True,True,True,True,False
56,True,True,True,True,True,False,True,True,True,True,True,False
57,True,True,True,True,True,False,True,True,True,True,True,False
58,True,True,True,True,True,False,True,True,True,True,True,False
59,True,True,True,True,True,False,True,True,True,True,True,False
60,True,True,True,True,True,False,True,True,True,True,True,False
61,True,True,True,True,True,False,True,True,True,True,True,False
62,True,True,True,True,True,False,True,True,True,True,True,False
63,True,True,True,True,True,False,True,True,True,True,True,False


In [46]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]', controles]

,index - id_pdc_itinerance,contact_operateur - id_station_itinerance,nom_enseigne - id_station_itinerance,coordonneesXY - id_station_itinerance,id_station_itinerance - id_pdc_itinerance,nom_station - id_station_itinerance,implantation_station - id_station_itinerance,nbre_pdc - id_station_itinerance,condition_acces - id_station_itinerance,horaires - id_station_itinerance,station_deux_roues - id_station_itinerance,adresse_station - coordonneesXY


In [47]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.666373, 47.527509]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [48]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.022143, 46.877453]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [49]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554, 48.728787]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Defauts operateur IZIVIA
Les défauts au nombre de 280 sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Izivia en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance, ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur un parking [-0.530598, 47.398424], on trouve 17 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué entre 1 et 3.

In [50]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'sav@izivia.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
239,12900,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2024-08-01,2024-08-01T08:32:01.605000+00:00
240,12901,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2024-08-01,2024-08-01T08:32:01.605000+00:00
241,12902,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2024-08-01,2024-08-01T08:32:01.605000+00:00
242,12903,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2024-08-01,2024-08-01T08:32:01.605000+00:00
243,12904,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2024-08-01,2024-08-01T08:32:01.605000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4084,102478,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1622,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1622,2024-08-01,2024-08-01T08:32:00.021000+00:00
4085,102479,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1631,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1631,2024-08-01,2024-08-01T08:32:00.021000+00:00
4086,102480,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1632,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1632,2024-08-01,2024-08-01T08:32:00.021000+00:00
4087,102481,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL AEROVILLE 95700 ROISSY-EN-FR...,FRURWPUNIB1711,AEROVILLE - TOKYO,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1711,2024-08-01,2024-08-01T08:32:00.021000+00:00


In [51]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-0.530598, 47.398424]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
239,12900,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2024-08-01,2024-08-01T08:32:01.605000+00:00
240,12901,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2024-08-01,2024-08-01T08:32:01.605000+00:00
241,12902,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2024-08-01,2024-08-01T08:32:01.605000+00:00
242,12903,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2024-08-01,2024-08-01T08:32:01.605000+00:00
243,12904,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2024-08-01,2024-08-01T08:32:01.605000+00:00
2589,60125,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372101,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372101,2024-08-01,2024-08-01T08:32:30.865000+00:00
2590,60126,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37211,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37211,2024-08-01,2024-08-01T08:32:30.865000+00:00
2591,60127,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372111,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372111,2024-08-01,2024-08-01T08:32:30.865000+00:00
2592,60128,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372121,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372121,2024-08-01,2024-08-01T08:32:30.865000+00:00
2593,60129,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37221,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37221,2024-08-01,2024-08-01T08:32:30.865000+00:00


### Defauts operateur IONITY
Les défauts (480 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Ionity en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur l'aire de Mornas [4.732409, 44.194832], on trouve 26 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué de 23 pour chaque station.

In [52]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1797,39666,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
1798,39674,info@ionity.eu,IONITY GMBH,"[0.547145, 49.614324]","Aire de services de Bolleville Est, A29, 76210...",FRIONE402300,IONITY Bolleville Est,Station dédiée à la recharge rapide,3,Accès libre,24/7,FALSE,FRIONE4023,2023-03-21,2024-01-19T07:47:22.735000+00:00
1799,39678,info@ionity.eu,IONITY GMBH,"[-1.420473, 43.586775]",Aire de Lacq Sud AUDEJOS A64-64170 Lacq,FRIONE402700,IONITY Lacq Audéjos Nord,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4027,2023-03-21,2024-01-19T07:47:22.735000+00:00
1800,39682,info@ionity.eu,IONITY GMBH,"[1.308488, 43.914901]","A7, Aire de Montélimar Est - 26780 Allan",FRIONE403500,IONITY Montélimar Est,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4035,2023-03-21,2024-01-19T07:47:22.735000+00:00
1801,39684,info@ionity.eu,IONITY GMBH,"[6.623059, 45.196766]","Autoroute de la Maurienne, A43 - Autoport Du F...",FRIONE403700,IONITY Freney,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4037,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094,40749,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440353,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440353,2024-08-01,2024-08-02T01:00:10.831000+00:00
2095,40750,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440371,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440371,2024-08-01,2024-08-02T01:00:10.831000+00:00
2096,40751,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440372,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440372,2024-08-01,2024-08-02T01:00:10.831000+00:00
2097,40752,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440373,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440373,2024-08-01,2024-08-02T01:00:10.831000+00:00


In [53]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1797,39666,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
1827,39809,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126401,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126401,2024-08-01,2024-08-02T01:00:10.831000+00:00
1828,39810,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126402,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126402,2024-08-01,2024-08-02T01:00:10.831000+00:00
1829,39811,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126404,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126404,2024-08-01,2024-08-02T01:00:10.831000+00:00
1830,39812,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126405,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126405,2024-08-01,2024-08-02T01:00:10.831000+00:00
1831,39813,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126451,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126451,2024-08-01,2024-08-02T01:00:10.831000+00:00
1832,39814,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126452,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126452,2024-08-01,2024-08-02T01:00:10.831000+00:00
1833,39815,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126453,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126453,2024-08-01,2024-08-02T01:00:10.831000+00:00


In [54]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[4.732409, 44.194832]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1825,39772,info@ionity.eu,IONITY GMBH,"[4.732409, 44.194832]","Aire de Mornas Village , A7, 84550 Mornas",FRIONE440300,IONITY Mornas Village,Station dédiée à la recharge rapide,16,Accès libre,24/7,FALSE,FRIONE4403,2023-03-21,2024-01-19T07:47:22.735000+00:00
2076,40731,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440301,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440301,2024-08-01,2024-08-02T01:00:10.831000+00:00
2077,40732,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440302,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440302,2024-08-01,2024-08-02T01:00:10.831000+00:00
2078,40733,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440303,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440303,2024-08-01,2024-08-02T01:00:10.831000+00:00
2079,40734,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440304,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440304,2024-08-01,2024-08-02T01:00:10.831000+00:00
2080,40735,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440305,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440305,2024-08-01,2024-08-02T01:00:10.831000+00:00
2081,40736,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440306,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440306,2024-08-01,2024-08-02T01:00:10.831000+00:00
2082,40737,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440307,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440307,2024-08-01,2024-08-02T01:00:10.831000+00:00
2083,40738,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440308,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440308,2024-08-01,2024-08-02T01:00:10.831000+00:00
2084,40739,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440309,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440309,2024-08-01,2024-08-02T01:00:10.831000+00:00


In [55]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1797,39666,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
1798,39674,info@ionity.eu,IONITY GMBH,"[0.547145, 49.614324]","Aire de services de Bolleville Est, A29, 76210...",FRIONE402300,IONITY Bolleville Est,Station dédiée à la recharge rapide,3,Accès libre,24/7,FALSE,FRIONE4023,2023-03-21,2024-01-19T07:47:22.735000+00:00
1799,39678,info@ionity.eu,IONITY GMBH,"[-1.420473, 43.586775]",Aire de Lacq Sud AUDEJOS A64-64170 Lacq,FRIONE402700,IONITY Lacq Audéjos Nord,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4027,2023-03-21,2024-01-19T07:47:22.735000+00:00
1800,39682,info@ionity.eu,IONITY GMBH,"[1.308488, 43.914901]","A7, Aire de Montélimar Est - 26780 Allan",FRIONE403500,IONITY Montélimar Est,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4035,2023-03-21,2024-01-19T07:47:22.735000+00:00
1801,39684,info@ionity.eu,IONITY GMBH,"[6.623059, 45.196766]","Autoroute de la Maurienne, A43 - Autoport Du F...",FRIONE403700,IONITY Freney,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4037,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094,40749,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440353,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440353,2024-08-01,2024-08-02T01:00:10.831000+00:00
2095,40750,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440371,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440371,2024-08-01,2024-08-02T01:00:10.831000+00:00
2096,40751,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440372,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440372,2024-08-01,2024-08-02T01:00:10.831000+00:00
2097,40752,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440373,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440373,2024-08-01,2024-08-02T01:00:10.831000+00:00


In [56]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.366838, 46.701436]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [57]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.523685, 48.9908]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4069,102463,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1311,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1311,2024-08-01,2024-08-01T08:32:00.021000+00:00
4070,102464,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1312,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1312,2024-08-01,2024-08-01T08:32:00.021000+00:00
4071,102465,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1321,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1321,2024-08-01,2024-08-01T08:32:00.021000+00:00
4072,102466,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1322,AEROVILLE - AFRICA LODGE,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1322,2024-08-01,2024-08-01T08:32:00.021000+00:00
4073,102467,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1411,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1411,2024-08-01,2024-08-01T08:32:00.021000+00:00
4074,102468,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1412,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1412,2024-08-01,2024-08-01T08:32:00.021000+00:00
4075,102469,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1421,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1421,2024-08-01,2024-08-01T08:32:00.021000+00:00
4076,102470,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1422,AEROVILLE - BALI MARKET,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1422,2024-08-01,2024-08-01T08:32:00.021000+00:00
4077,102471,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1511,AEROVILLE - NORDIC CHIC,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1511,2024-08-01,2024-08-01T08:32:00.021000+00:00
4078,102472,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1512,AEROVILLE - NORDIC CHIC,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1512,2024-08-01,2024-08-01T08:32:00.021000+00:00


### Defauts operateur BOUYGUES
Les défauts (286 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes).

Les défauts sont majoritairement associés à l'enseigne Ecocharge77 pour laquelle on trouve des stations en doublon entre Bouygues et AlizeCharge. Ce défaut semble être lié à un transfert de bornes d'un opérateur à un autre sans que les anciennes stations aient été supprimées.

Par exemple, pour la coordonnées [3.438183,48.694653], on trouve quatre points de recharge associés à deux stations (une est opérée par Bouygues et l'autre par AlizeCharge).

In [78]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'a.buffler@bouygues-es.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3307,87831,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710011,2024-07-03,2024-07-03T14:44:38.430000+00:00
3308,87832,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710012,2024-07-03,2024-07-03T14:44:38.430000+00:00
3309,87833,a.buffler@bouygues-es.com,Ecocharge77,"[2.938246,48.675091]",Avenue du général Leclerc,FRS77P77031A,Bernay-Vilbert - Salle des fêtes,Voirie,2,Accès libre,24/7,false,FRS77EEC771011,2024-07-03,2024-07-03T14:44:38.430000+00:00
3310,87836,a.buffler@bouygues-es.com,Ecocharge77,"[2.938246,48.675091]",Avenue du général Leclerc,FRS77P77031A,Bernay-Vilbert - Salle des fêtes,Voirie,2,Accès libre,24/7,false,FRS77EEC771012,2024-07-03,2024-07-03T14:44:38.430000+00:00
3311,87839,a.buffler@bouygues-es.com,Ecocharge77,"[2.743907,49.025669]",route de Juilly,FRS77P77308A,Montge-en-Goele,Voirie,2,Accès libre,24/7,false,FRS77EEC7710311,2024-07-03,2024-07-03T14:44:38.430000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3588,88268,a.buffler@bouygues-es.com,Ecocharge77,"[2.859685,48.926731]",Place Jean Jaurès,FRS77P77276A,Mareuil-les-Meaux,Voirie,2,Accès libre,24/7,false,FRS77EEC779412,2024-07-03,2024-07-03T14:44:38.430000+00:00
3589,88275,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779811,2024-07-03,2024-07-03T14:44:38.430000+00:00
3590,88276,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779812,2024-07-03,2024-07-03T14:44:38.430000+00:00
3591,88277,a.buffler@bouygues-es.com,Ecocharge77,"[2.985625,48.941795]",place de la mairie,FRS77P77300A,Montceaux-les-Meaux,Voirie,2,Accès libre,24/7,false,FRS77EEC779911,2024-07-03,2024-07-03T14:44:38.430000+00:00


In [79]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.850102,48.349255]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3191,87665,support@alizecharge.fr,Ecocharge77,"[2.850102,48.349255]","Rue Grande, 77250 MORET-LOING-ORVANNE",FRS77E77316001,MORET-LOING-ORVANNE - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773160011,2024-07-26,2024-07-26T08:27:18.771000+00:00
3192,87666,support@alizecharge.fr,Ecocharge77,"[2.850102,48.349255]","Rue Grande, 77250 MORET-LOING-ORVANNE",FRS77E77316001,MORET-LOING-ORVANNE - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773160012,2024-07-26,2024-07-26T08:27:18.771000+00:00
3589,88275,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779811,2024-07-03,2024-07-03T14:44:38.430000+00:00
3590,88276,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779812,2024-07-03,2024-07-03T14:44:38.430000+00:00


In [80]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[3.438183,48.694653]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3183,87653,support@alizecharge.fr,Ecocharge77,"[3.438183,48.694653]","Route De Brangis, 77151 MONTCEAUX-LES-PROVINS",FRS77E77301001,MONTCEAUX-LES-PROVINS - Route De Brangis,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773010011,2024-07-26,2024-07-26T08:27:18.771000+00:00
3184,87654,support@alizecharge.fr,Ecocharge77,"[3.438183,48.694653]","Route De Brangis, 77151 MONTCEAUX-LES-PROVINS",FRS77E77301001,MONTCEAUX-LES-PROVINS - Route De Brangis,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773010012,2024-07-26,2024-07-26T08:27:18.771000+00:00
3307,87831,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710011,2024-07-03,2024-07-03T14:44:38.430000+00:00
3308,87832,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710012,2024-07-03,2024-07-03T14:44:38.430000+00:00


### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [58]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
44,2015,support@alizecharge.fr,ADP Groupe,"[2.366988,48.729465]","Rue de Berlin, 94390 Paray-Vieille-Poste",FRADPE91479015,ORY - Parking P4b Rouge - Niveau Terrasse,Voirie,12,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790151,2024-07-26,2024-07-26T08:27:18.771000+00:00
45,2019,support@alizecharge.fr,ADP Groupe,"[2.366988,48.729465]","Rue de Berlin, 94390 Paray-Vieille-Poste",FRADPE91479015,ORY - Parking P4b Rouge - Niveau Terrasse,Parking public,12,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790152,2024-07-26,2024-07-26T08:27:18.771000+00:00
46,2020,support@alizecharge.fr,ADP Groupe,"[2.366988,48.729465]","Rue de Berlin, 94390 Paray-Vieille-Poste",FRADPE91479015,ORY - Parking P4b Rouge - Niveau Terrasse,Parking public,12,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790153,2024-07-26,2024-07-26T08:27:18.771000+00:00
47,2021,support@alizecharge.fr,ADP Groupe,"[2.366988,48.729465]","Rue de Berlin, 94390 Paray-Vieille-Poste",FRADPE91479015,ORY - Parking P4b Rouge - Niveau Terrasse,Parking public,12,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790154,2024-07-26,2024-07-26T08:27:18.771000+00:00
48,2022,support@alizecharge.fr,ADP Groupe,"[2.366988,48.729465]","Rue de Berlin, 94390 Paray-Vieille-Poste",FRADPE91479015,ORY - Parking P4b Rouge - Niveau Terrasse,Parking public,12,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790155,2024-07-26,2024-07-26T08:27:18.771000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792,93222,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE94,Shell Fenioux Est,Voirie,8,Accès réservé,24/7,false,FRSHEE944,2024-08-01,2024-08-02T01:00:10.831000+00:00
3793,93223,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE951,2024-08-01,2024-08-02T01:00:10.831000+00:00
3794,93224,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE952,2024-08-01,2024-08-02T01:00:10.831000+00:00
3795,93225,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE953,2024-08-01,2024-08-02T01:00:10.831000+00:00


### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [59]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
54,2036,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901810,2024-07-26,2024-07-26T08:27:18.771000+00:00
55,2037,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901811,2024-07-26,2024-07-26T08:27:18.771000+00:00
56,2038,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901812,2024-07-26,2024-07-26T08:27:18.771000+00:00
57,2039,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901813,2024-07-26,2024-07-26T08:27:18.771000+00:00
58,2040,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901814,2024-07-26,2024-07-26T08:27:18.771000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,99464,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550401,2024-07-26,2024-07-26T08:27:18.771000+00:00
3980,99465,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550402,2024-07-26,2024-07-26T08:27:18.771000+00:00
3981,99466,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550403,2024-07-26,2024-07-26T08:27:18.771000+00:00
3982,99467,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550404,2024-07-26,2024-07-26T08:27:18.771000+00:00


### Coherence adresse - coordonnees
- 2337 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [60]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
20,290,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:00,false,FR3R3E10000849681,2024-06-26,2024-06-26T08:32:08.391000+00:00
21,291,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:01,false,FR3R3E10000849682,2024-06-26,2024-06-26T08:32:08.391000+00:00
22,292,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849691,2024-06-26,2024-06-26T08:32:08.391000+00:00
23,293,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849692,2024-06-26,2024-06-26T08:32:08.391000+00:00
24,318,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849851,2024-06-26,2024-06-26T08:32:08.391000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4601,107768,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2024-01-19T07:46:24.141000+00:00
4602,107779,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
4603,107780,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
4604,107781,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [61]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4602,107779,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
4603,107780,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
4604,107781,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00
4605,107782,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [62]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4602,107779,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
4604,107781,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [63]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4603,107780,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
4605,107782,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


### Cohérence station - coordonnées
- 218 pdc sont associés à des stations avec plusieurs coordonnées (majorité AlizeCharge)

In [64]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1,129,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-01-19T07:46:04.014000+00:00
2,130,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-01-19T07:46:04.014000+00:00
3,131,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-01-19T07:46:04.014000+00:00
4,132,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-01-19T07:46:04.014000+00:00
5,133,acelec@acelec-france.com,HOTEL KYRIAD PARIS BEZONS,"[2.212336,48.931356]",80 Av. Gabriel Péri 95870 BEZONS,FR073PKYRIADBEZONS,HOTEL KYRIAD PARIS BEZONS,Parking privé à usage public,1,Accès libre,Mo-Fr 08:00-19:01,false,FR073EJJYD51230,2022-07-21,2024-01-19T07:46:04.014000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,99464,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550401,2024-07-26,2024-07-26T08:27:18.771000+00:00
3980,99465,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550402,2024-07-26,2024-07-26T08:27:18.771000+00:00
3981,99466,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550403,2024-07-26,2024-07-26T08:27:18.771000+00:00
3982,99467,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550404,2024-07-26,2024-07-26T08:27:18.771000+00:00


In [65]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRTLSE31555040', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2669,99464,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550401,2024-07-26,2024-07-26T08:27:18.771000+00:00
2670,99465,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550402,2024-07-26,2024-07-26T08:27:18.771000+00:00
2671,99466,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550403,2024-07-26,2024-07-26T08:27:18.771000+00:00
2672,99467,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550404,2024-07-26,2024-07-26T08:27:18.771000+00:00
2673,99468,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550405,2024-07-26,2024-07-26T08:27:18.771000+00:00


In [66]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRFR1PEHNKRRPQNR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [67]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1,129,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-01-19T07:46:04.014000+00:00
2,130,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-01-19T07:46:04.014000+00:00
3,131,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-01-19T07:46:04.014000+00:00
4,132,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-01-19T07:46:04.014000+00:00
6,134,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2024-01-19T07:46:04.014000+00:00
7,135,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2024-01-19T07:46:04.014000+00:00
8,136,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2024-01-19T07:46:04.014000+00:00
9,137,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2024-01-19T07:46:04.014000+00:00
10,138,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2024-01-19T07:46:04.014000+00:00
11,139,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2024-01-19T07:46:04.014000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [68]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [69]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.08745032, 48.40887156]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [70]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [71]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [72]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2293,52838,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.719183,45.152774]","Rue de Provence, 38130 ECHIROLLES",FRM38E38151001,ECHIROLLES - Parking Relais Échirolles - Gare,Voirie,5,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381510016,2024-07-26,2024-07-26T08:27:18.771000+00:00
2296,52852,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.72939,45.18938]","Place Jean Achard, 38000 GRENOBLE",FRM38E38185001,GRENOBLE - Place Jean Achard,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850011,2024-07-26,2024-07-26T08:27:18.771000+00:00
2297,52853,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.72939,45.18938]","Place Jean Achard, 38000 GRENOBLE",FRM38E38185001,GRENOBLE - Place Jean Achard,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850012,2024-07-26,2024-07-26T08:27:18.771000+00:00
2298,52854,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.72949,45.18938]","Place Jean Achard (CITIZ), 38000 GRENOBLE",FRM38E38185001,GRENOBLE - Place Jean Achard (CITIZ),Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850013,2024-07-26,2024-07-26T08:27:18.771000+00:00
2299,52858,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.704011,45.1844]","Rue Victor Lastella, 38000 GRENOBLE",FRM38E38185005,GRENOBLE - Rue Victor Lastella - Chambre des M...,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850051,2024-07-26,2024-07-26T08:27:18.771000+00:00
2300,52859,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.704011,45.1844]","Rue Victor Lastella, 38000 GRENOBLE",FRM38E38185005,GRENOBLE - Rue Victor Lastella - Chambre des M...,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850052,2024-07-26,2024-07-26T08:27:18.771000+00:00
2301,52860,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.705,45.1844]","Chambre des Métiers (CITIZ), 38000 GRENOBLE",FRM38E38185005,GRENOBLE - Rue Victor Lastella - Chambre des M...,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850053,2024-07-26,2024-07-26T08:27:18.771000+00:00
2302,52869,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.729505,45.18551]","Place André Malraux, 38000 GRENOBLE",FRM38E38185013,GRENOBLE - Place André Malraux - CCI,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850131,2024-07-26,2024-07-26T08:27:18.771000+00:00
2303,52870,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.729,45.18551]","Place André Malraux - CCI (CITIZ), 38000 GREN...",FRM38E38185013,GRENOBLE - Place André Malraux - CCI (CITIZ),Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850132,2024-07-26,2024-07-26T08:27:18.771000+00:00
2304,52871,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.729505,45.18551]","Place André Malraux, 38000 GRENOBLE",FRM38E38185013,GRENOBLE - Place André Malraux - CCI,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850133,2024-07-26,2024-07-26T08:27:18.771000+00:00


### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [73]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
411,25857,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
412,25858,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
413,25859,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-01-19T07:50:45.527000+00:00
2259,51326,contact@rirodo.fr,LAST MILES SOLUTIONS,"[5.39, 43.27]",20 Rue Negresko,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRLMSE1897228,2023-09-06,2024-01-19T07:49:16.739000+00:00
2749,74468,contact@rirodo.fr,MONTA,"[5.39, 43.27]",20 Rue NEGRESKO,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRRIRE2804856,2023-09-06,2024-01-19T07:49:16.739000+00:00


In [74]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
24957,24977,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
24959,24979,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
24961,24981,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU3,2022-08-25,2024-01-19T07:49:57.766000+00:00
25837,25857,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
25838,25858,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
25839,25859,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-01-19T07:50:45.527000+00:00


In [75]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
24957,24977,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
24959,24979,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
25837,25857,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
25838,25858,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
29246,29266,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS1,2022-10-04,2024-01-19T07:49:31.469000+00:00
29248,29268,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS2,2022-10-04,2024-01-19T07:49:31.469000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [76]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [77]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
101370,104429,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17161,2023-07-06,2024-07-10T14:10:45.800000+00:00
101371,104430,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17162,2023-07-06,2024-07-10T14:10:45.800000+00:00
101372,104431,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17163,2023-07-06,2024-07-10T14:10:45.800000+00:00
101373,104432,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17164,2023-07-06,2024-07-10T14:10:45.800000+00:00
101374,104433,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17165,2023-07-06,2024-07-10T14:10:45.800000+00:00
101375,104434,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17166,2023-07-06,2024-07-10T14:10:45.800000+00:00
101376,104435,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17167,2023-07-06,2024-07-10T14:10:45.800000+00:00
